In [1]:
import os
import numpy as np
import pandas as pd
import datetime as dt
import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy import interpolate
from scipy.stats import norm
import torch
from itertools import chain
from torch.utils.data import TensorDataset, DataLoader

from lib.encoder_back_gru import Encoder_Back_GRU
import lib.Metrics as Metrics
from lib.hhs_data_builder import DataConstructor, convert_to_torch
# from lib.encoders import 
from torch import nn
import lib.encoders as encoders
from torchdiffeq import odeint
import lib.train_functions as train_functions
import lib.utils as utils
from lib.UONN import FaFp
from lib.models import Decoder, Fa, Fp
from torch.distributions import Categorical, Normal
from lib.VAE import VAE
from filelock import FileLock

In [2]:
torch.set_num_threads(1)

batch_size = 32
window_size = 28
n_regions = 49
lr = 1e-3
dtype = torch.float32
gamma = 28


t = torch.arange(window_size + gamma + 1, dtype=dtype)/7
eval_pts = np.arange(0,t.shape[-1], 7)


In [3]:
batch_size = 32
window_size = 28
n_regions = 49
n_qs = 9
latent_dim = 8
lr = 1e-3
dtype = torch.float32
gamma = 28
n_samples = 64

mydict =   {'Full_FaFp':['UONN', {'nll':True, 'kl_z':True, 'kl_p':True,  'Fa_norm':True,  'reg_loss':True,  'anneal':True}],
            'Half_FaFp':['UONN', {'nll':True, 'kl_z':True, 'kl_p':False, 'Fa_norm':True,  'reg_loss':False, 'anneal':True}],
            'Full_Fp':  ['CONN', {'nll':True, 'kl_z':True, 'kl_p':True,  'Fa_norm':False, 'reg_loss':True,  'anneal':True}],
            'Full_Fa':  ['CONN', {'nll':True, 'kl_z':True, 'kl_p':False, 'Fa_norm':False, 'reg_loss':False, 'anneal':True}],
            'Fa':       ['SONN', {'nll':True, 'kl_z':True, 'kl_p':False, 'Fa_norm':False, 'reg_loss':False, 'anneal':True}],
           }

t = torch.arange(window_size + gamma + 1, dtype=dtype)/7
eval_pts = np.arange(0,t.shape[-1], 7)
epochs = 250
q_sizes = [256, 128]
ff_sizes = [128, 64]
n_qs = 5
latent_dim = 6
epochs = 250
region = 'state'
started_file_path = "HHS_started.txt"
ode_name = 'UONN'
for num in range(10, 16):
    for test_season in [2015, 2016, 2017, 2018]:
        for key in list(mydict.keys()):
            pass


In [4]:
num = 1
test_season = 2015



In [5]:
_data = DataConstructor(test_season=test_season, region = region, window_size=window_size, n_queries=n_qs, fill_1 = True, gamma=gamma)
x_train, y_train, x_test, y_test, scaler = _data(run_backward=True, no_qs_in_output=True)
train_loader, x_test, y_test = convert_to_torch(x_train, y_train, x_test, y_test, batch_size=32, shuffle=True, dtype=dtype) 

In [19]:
ode_name = 'SONN'
losses = mydict[key][1]
ode = {'CONN':Fp, 'UONN':FaFp, 'SONN':Fa}[ode_name]
num = 'test'
file_prefix = f'weights/{ode_name}_{key}/{region}_{num}_{test_season}_'
model = VAE(Encoder_Back_GRU, ode, Decoder, n_qs, latent_dim, n_regions, q_sizes=q_sizes, ff_sizes=ff_sizes, file_prefix=file_prefix)
model.setup_training(lr=lr)

model.pre_train(train_loader, 3)

Training: 100%|██████████| 55/55 [00:03<00:00, 16.27it/s, Epoch=3, KL_z=4.95]


NameError: name 'torch' is not defined

In [20]:
num = 12
test_season = 2015

key = 'Fa'

t = torch.arange(window_size + gamma + 1, dtype=dtype)/7
eval_pts = torch.arange(57)

In [21]:
new_lr = 1e-3
for param_group in model.optimizer.param_groups:
    param_group['lr'] = new_lr


In [ ]:
model.train(train_loader, t, epochs, losses, eval_pts, grad_lim=650, checkpoint=True, track_norms=True, norm_file=f'{file_prefix}norms.txt')

Training 81:   7%|▋         | 4/55 [00:07<01:38,  1.94s/it, loss=-.224, kl_w=0.965, nll=-.227, kl_latent=0.0025, grad_norm=1.47] 

In [ ]:
model.train(train_loader, t, epochs, losses, eval_pts, grad_lim=300, n_samples=32, checkpoint=False, track_norms = False, norm_file = 'grad_norms.txt'

In [ ]:
    for test_season in [2015, 2016, 2017, 2018]:
        _data = DataConstructor(test_season=test_season, region = region, window_size=window_size, n_queries=n_qs, gamma=gamma)
        x_train, y_train, x_test, y_test, scaler = _data(run_backward=True, no_qs_in_output=True)
        train_loader, x_test, y_test = convert_to_torch(x_train, y_train, x_test, y_test, batch_size=32, shuffle=True, dtype=dtype) 
                
        for num in range(10, 16):
            for key in list(mydict.keys()):
                try:
                    file_prefix = f'weights/{ode_name}_{key}/{region}_{num}_{test_season}_'
                    ode_name = mydict[key][0]
                    losses = mydict[key][1]
                    ode = {'CONN':Fp, 'UONN':FaFp, 'SONN':Fa}[ode_name]

                    model = VAE(Encoder_Back_GRU, ode, Decoder, n_qs, latent_dim, n_regions, q_sizes=q_sizes, ff_sizes=ff_sizes, file_prefix=file_prefix)
                    model.load(checkpoint=True)

                    y_pred = model(x_test, t, n_samples=128)
                    y_pr = y_pred.detach().numpy() * scaler.values[np.newaxis, np.newaxis, np.newaxis, :]
                    y_te = y_test.detach().numpy() * scaler.values[np.newaxis, np.newaxis, :]

                    pred_mean = y_pr.mean(1)
                    pred_std = y_pr.std(1)

                    results = {'ground_truth':y_te.tolist(),
                            'mean':pred_mean.tolist(),
                            'std': pred_std.tolist(),
                            }

                    skills = pd.DataFrame(index = [7,14,21,28], columns = [2015,2016,2017,2018], dtype=float)
                    for g_tr, g in zip([7,14,21,28], [-22,-15,-8,-1]):
                        skills.loc[g_tr, test_season] = np.exp(np.mean(Metrics.mb_log(y_te[:, g, :], pred_mean[:, g, :], pred_std[:, g, :]))).astype(float)
                    print(file_prefix)
                    print(np.exp(np.mean(np.log(skills), 1)))
                except:
                    print(file_prefix)
                    print('failed')

In [ ]:
# typo where Full_Fa is actually Half_Fp

In [ ]:
num = 1


q_sizes = [128, 64,32]
ff_sizes = [64]
n_qs = 10
latent_dim = 8
epochs = 100
region = 'state'
for num in range(2,6):
    for test_season in [2015, 2016, 2017, 2018]:
        for ode_name in ['UONN']:
            file_prefix = f'weights/{ode_name}/{region}_{num}_{test_season}_'
                    
            print(ode_name, test_season, num)
            ode = {'CONN':Fp, 'UONN':FaFp, 'SONN':Fa}[ode_name]

            _data = DataConstructor(test_season=test_season, region = region, window_size=window_size, n_queries=n_qs, gamma=gamma)
            x_train, y_train, x_test, y_test, scaler = _data(run_backward=True, no_qs_in_output=True)
            train_loader, x_test, y_test = convert_to_torch(x_train, y_train, x_test, y_test, batch_size=32, shuffle=True, dtype=dtype)  

            model = VAE(Encoder_Back_GRU, ode, Decoder, n_qs, latent_dim, n_regions, q_sizes=q_sizes, ff_sizes=ff_sizes, file_prefix=file_prefix)
            model.setup_training(lr=lr)

            model.pre_train(train_loader, 3)
            model.train(train_loader, t, epochs, eval_pts, grad_lim=650, checkpoint=True, track_norms=True, norm_file=f'{file_prefix}norms.txt')
            model.save()

In [ ]:
num = 2
test_season = 2018
ode_name = 'UONN'

file_prefix = f'weights/{ode_name}/{region}_{num}_{test_season}_'
        
print(ode_name, test_season, num)
ode = {'CONN':Fp, 'UONN':FaFp, 'SONN':Fa}[ode_name]

_data = DataConstructor(test_season=test_season, region = region, window_size=window_size, n_queries=n_qs, gamma=gamma)
x_train, y_train, x_test, y_test, scaler = _data(run_backward=True, no_qs_in_output=True)
train_loader, x_test, y_test = convert_to_torch(x_train, y_train, x_test, y_test, batch_size=32, shuffle=True, dtype=dtype)  

In [ ]:
model = VAE(Encoder_Back_GRU, ode, Decoder, n_qs, latent_dim, n_regions, q_sizes=q_sizes, ff_sizes=ff_sizes, file_prefix=file_prefix)
model.setup_training(lr=lr)

model.pre_train(train_loader, 3)
model.train(train_loader, t, epochs, eval_pts, grad_lim=650, checkpoint=True, track_norms=True, norm_file=f'{file_prefix}norms.txt')
# model.save()

In [ ]:
num = 1


q_sizes = [128, 64,32]
ff_sizes = [64]
n_qs = 10
latent_dim = 8
epochs = 100
region = 'state'
for num in range(2,3):
    for test_season in [2016]:
        for ode_name in ['CONN']:
            

            file_prefix = f'weights/{ode_name}/{region}_{num}_{test_season}_'
            run = False
                    
            print(ode_name, test_season, num)
            ode = {'CONN':Fp, 'UONN':FaFp, 'SONN':Fa}[ode_name]

            _data = DataConstructor(test_season=test_season, region = region, window_size=window_size, n_queries=n_qs, gamma=gamma)
            x_train, y_train, x_test, y_test, scaler = _data(run_backward=True, no_qs_in_output=True)
            train_loader, x_test, y_test = convert_to_torch(x_train, y_train, x_test, y_test, batch_size=32, shuffle=True, dtype=dtype)  



In [ ]:
model = VAE(Encoder_Back_GRU, ode, Decoder, n_qs, latent_dim, n_regions, q_sizes=q_sizes, ff_sizes=ff_sizes, file_prefix=file_prefix)
model.load()

y_pred = model(x_test, t, n_samples=128)
y_pr = y_pred.detach().numpy() * scaler.values[np.newaxis, np.newaxis, np.newaxis, :]
y_te = y_test.detach().numpy() * scaler.values[np.newaxis, np.newaxis, :]

pred_mean = y_pr.mean(1)
pred_std = y_pr.std(1)

results = {'ground_truth':y_te.tolist(),
        'mean':pred_mean.tolist(),
        'std': pred_std.tolist(),
        }

skills = pd.DataFrame(index = [7,14,21,28], columns = [2015,2016,2017,2018], dtype=float)
for g_tr, g in zip([7,14,21,28], [-22,-15,-8,-1]):
    skills.loc[g_tr, test_season] = np.exp(np.mean(Metrics.mb_log(y_te[:, g, :], pred_mean[:, g, :], pred_std[:, g, :]))).astype(float)
print(np.exp(np.mean(np.log(skills), 1)))

In [ ]:
state_codes = {'AK':'Alaska','AL':'Alabama','AR':'Arkansas','AZ':'Arizona','CA':'California','CO':'Colorado',
                'CT':'Connecticut','DE':'Delaware','DC':'District of Columbia','GA':'Georgia', 'HI':'Hawaii',
                'ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky',
                'LA':'Louisiana','ME':'Maine','MD':'Maryland','MI':'Michigan','MN':'Minnesota','MS':'Mississippi',
                'MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey',
                'NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
                'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota',
                'TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington',
                'WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming'}

In [ ]:
code = 'CA'
state_num = np.where([code == state_code for state_code in list(state_codes.keys())])[0][0]

plt.plot(np.asarray(results['ground_truth'])[:, -1, state_num], color='black')
plt.plot(np.asarray(results['mean'])[:, -21, state_num], color='red')
plt.fill_between(np.arange(np.asarray(results['mean']).shape[0]),
                 (np.asarray(results['mean'])-np.asarray(results['std']))[:, -21, state_num],
                 (np.asarray(results['mean'])+np.asarray(results['std']))[:, -21, state_num], 
                 color='red', linewidth=0, alpha=0.3)

In [ ]:
np.arange(np.asarray(results['mean']).shape[0])

In [46]:
import numpy as np

def function(queries, N):
    array = np.zeros(N)
    return_list = []
    for inputs in queries:
        setget = inputs[0]

        if setget == 1:
            array[inputs[1]] = 1

        if setget == 2:
            pos = inputs[1]

            try:
                return_list.append(np.where(array[pos:] == 1)[0][0] + pos)
            except:
                return_list.append(-1)
    return return_list

In [47]:
queries = [[2, 3], [1, 2], [2, 1], [2, 3], [2, 2]]
N = 5

In [48]:
function(queries, N)

[-1, 2, -1, 2]

In [132]:
import numpy as np
def function2(A, N):
    return_arr = []
    for l1 in range(N):
        for r1 in range(l1+1, N+1):
            sub = np.mean(A[l1:r1])
            
            sub2r = np.sum(A[r1:])
            sub2l = np.sum(A[:l1])
            
            if np.isnan(sub2r):
                sub2r=0
                
            if np.isnan(sub2l):
                sub2l=0
            if len(A[r1:])+len(A[:l1]) == 0:
                sub2 = 0
            else:
                sub2 = (sub2l+sub2r)/(len(A[r1:])+len(A[:l1]))
            if sub > sub2:
                return_arr.append([l1+1, r1])
    return return_arr

In [133]:
A = [3, 4, 2]
N = 3

In [134]:
function2(A, N)

[[1, 2], [1, 3], [2, 2]]